# Profitability Predictions for iOS and Android Apps

A company that develops free apps generates most of its revenue from in-app ads. The more users see and interact with the ads, the better. Our task is to determine what kinds of apps are likely to attract the most users.

As of September 2018, there were approximately 2 million apps available for download on the App Store and 2.1 million available on Google Play. We'll only use a subset of that data here.

Luckily, Kaggle hosts two datasets that will be of use here.

The [Google Play Store Apps dataset](https://www.kaggle.com/lava18/google-play-store-apps/home) contains data about approximately ten thousand Android apps. The data was collected in August 2018.

The [Mobile App Store dataset](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home) contains data about approximately seven thousand iOS apps. The data was collected in July 2017.

## Open the datasets and save each as a list of lists

In [1]:
from csv import reader

opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
ios_apps = list(read_file)
ios_header = ios_apps[0]
ios_data = ios_apps[1:]

opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
android_apps = list(read_file)
android_header = android_apps[0]
android_data = android_apps[1:]

## explore_data()

The ```explore_data()``` function formats our data in a way that is much easier to read. It takes four parameters -- ```dataset```, which should be a list of lists, ```start``` and ```end```, which represents the indices of a slice from the dataset, and ```rows_and_columns```.

The ```explore_data()``` function slices the dataset using ```dataset[start:end]``` then loops through the slice to print each row followed by a new line. When ```rows_and_columns``` is set to ```True```, the function also prints the number of rows and columns in the dataset.

Here we'll use the ```explore_data()``` function to print the first three rows of each dataset and the number of rows and columns.

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
        
explore_data(ios_data, 0, 3, True)
print('\n')
explore_data(android_data, 0, 3, True)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1,

Printing the header rows gives us the column names for each dataset. 

Descriptions of the iOS column names can be found [here](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home)

Descriptions of the Android column names can be found [here](https://www.kaggle.com/lava18/google-play-store-apps/home)

In [3]:
print(ios_header)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


In [4]:
print(android_header)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


# Data Cleaning

## Removing inaccurate data

Both the Android and iOS datasets have dedicated discussion pages at Kaggle.com. Looking at the discussion for the Android Google Play data, a user has identified a row with missing data that causes a column shift. This error is in row 10472 when the header is removed. We'll just remove this row.

In [5]:
del android_data[10472]

## Removing duplicate entries
The Android dataset also has a number of duplicate entries. The following code determines how many duplicates there are and finds some examples. 

We create two empty lists: ```duplicate_apps``` and ```unique_apps```. Looping through the Android data, if a name already appears in ```unique_apps```, the name is added to the ```duplicate_apps``` list. Otherwise, the app name is added to the ```unique_apps``` list.

In [6]:
duplicate_apps = []
unique_apps = []

for app in android_data:
    name = app[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)
        
print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:5])

Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings']


We need to remove duplicate entries but don't want to do so randomly. We'll assume the entry with the most user ratings is the most recent. The following code removes all duplicate entries but the ones with the most user ratings. 

First, we create a dictionary where each key is a unique app and the value is the highest number of reviews for the app.

In [7]:
reviews_max = {}

for app in android_data:
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
    if name not in reviews_max:
        reviews_max[name] = n_reviews


We can confirm we have the correct number of apps by comparing the length of ```reviews_max``` to the length of our original dataset less the 1181 duplicate apps.

In [8]:
print(len(reviews_max))
print(len(android_data) - 1181)

9659
9659


We can use the dictionary created above to remove the duplicate entries. We create two empty lists -- one to store our clean dataset and one to store a list of apps already added to the new dataset. If the number of reviews of an app equals that app's value in the ```reviews_max``` dictionary and the app's name is not in the ```already_added``` list, the app is added to the ```android_clean``` dataset and the app's name is added to the ```already_added``` list. 

In [9]:
android_clean = []
already_added = []

for app in android_data:
    name = app[0]
    n_reviews = float(app[3])
    if n_reviews == reviews_max[name] and name not in already_added: 
        android_clean.append(app)
        already_added.append(app[0])        

The ```android_clean``` list should contain 9659 values.

In [10]:
print(len(android_clean))

9659


## Remove non-English apps

ASCII codes for characters commonly used in English range from 0 to 127. So any app whose name contains a character whose code is greater than 127 is possibly a non-English app. The ```is_english()``` function returns ```False``` if any character in a string is not a common English character and returns ```True``` otherwise.

In [11]:
def is_english(string):
    for char in string:
        if ord(char) > 127:
            return False
    return True

However, some English apps do have names that use characters outside the normal ASCII range, such as ™ or emoticons.

In [12]:
print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
False
False
False


A better idea might be to only eliminate apps that have three or more characters outside the normal range. 

In [13]:
def is_english(string):
    chars = 0
    for char in string:
        if ord(char) > 127:
            chars += 1
            
    if chars >= 3:
        return False
    return True

In [14]:
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

False
True
True


We'll use the ```is_english()``` function to filter out non-English apps. We'll loop through the Android and iOS app datasets and add English apps to a new list.

In [15]:
android_english = []
ios_english = []

for app in android_clean:
    name = app[0]
    if is_english(name):
        android_english.append(app)
        
for app in ios_data:
    name = app[1]
    if is_english(name):
        ios_english.append(app)

## Isolating free apps
We only want to look at free apps. We'll loop through the ```android_english``` and ```ios_english``` datasets and append any free apps to the ```android_free``` or ```ios_free``` lists.

In [17]:
android_free = []
ios_free = []

for app in android_english:
    price = app[7]
    if price == '0':
        android_free.append(app)
        
for app in ios_english:
    price = app[4]
    if price == '0.0':
        ios_free.append(app)
    

## Determine most common genres in each market
We'll start by creating a frequency table of genres in each market. 
The ```freq_table()``` function takes two parameters: dataset and index of the column we want to create a frequency table for. It starts with an empty dictionary. We loop through both datasets. If a key is not already in the ```ft``` dictionary, it is added. Otherwise, the value will be incremented by one. Frequency table values will be converted to a percentage of total number of apps.

In [22]:
def freq_table(dataset, index):
    ft = {}
    total = 0
    
    for key in dataset:
        total += 1
        value = key[index]
        if value in ft:
            ft[value] += 1
        else:
            ft[value] = 1

    percentages = {}
    for each in ft:
        percent = (ft[each] / total) * 100
        percentages[each] = percent
        
    return percentages

print(freq_table(android_free, 1))
print('\n')
print(freq_table(ios_free, -5))

{'ART_AND_DESIGN': 0.6442133815551537, 'TRAVEL_AND_LOCAL': 2.3395117540687163, 'SPORTS': 3.390596745027125, 'PHOTOGRAPHY': 2.949819168173599, 'BUSINESS': 4.599909584086799, 'FAMILY': 18.942133815551536, 'NEWS_AND_MAGAZINES': 2.802893309222423, 'DATING': 1.8648282097649187, 'GAME': 9.697106690777577, 'PERSONALIZATION': 3.322784810126582, 'EVENTS': 0.7120253164556962, 'BEAUTY': 0.599005424954792, 'FOOD_AND_DRINK': 1.2432188065099457, 'FINANCE': 3.7070524412296564, 'LIFESTYLE': 3.887884267631103, 'AUTO_AND_VEHICLES': 0.9267631103074141, 'BOOKS_AND_REFERENCE': 2.1360759493670884, 'HEALTH_AND_FITNESS': 3.0854430379746836, 'EDUCATION': 1.164104882459313, 'WEATHER': 0.7911392405063291, 'COMICS': 0.6103074141048824, 'SHOPPING': 2.2490958408679926, 'TOOLS': 8.453887884267631, 'MAPS_AND_NAVIGATION': 1.3901446654611211, 'PARENTING': 0.6555153707052441, 'LIBRARIES_AND_DEMO': 0.9380650994575045, 'HOUSE_AND_HOME': 0.8024412296564195, 'PRODUCTIVITY': 3.899186256781193, 'COMMUNICATION': 3.232368896925

The ```display_table()``` function sorts our results and formats them nicely. This function takes in two parameters: dataset and index. First, the ```display_table()``` function uses the ```freq_table()``` to generate a frequency table. We need to sort our results by value but the built-in ```sorted()``` function sorts dictionaries by key. So first we must convert the dictionary to a list of tuples.

In [23]:
def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])

In [25]:
print(display_table(android_free, 1))
print('\n')
print(display_table(ios_free, -5))

FAMILY : 18.942133815551536
GAME : 9.697106690777577
TOOLS : 8.453887884267631
BUSINESS : 4.599909584086799
PRODUCTIVITY : 3.899186256781193
LIFESTYLE : 3.887884267631103
FINANCE : 3.7070524412296564
MEDICAL : 3.5375226039783
SPORTS : 3.390596745027125
PERSONALIZATION : 3.322784810126582
COMMUNICATION : 3.2323688969258586
HEALTH_AND_FITNESS : 3.0854430379746836
PHOTOGRAPHY : 2.949819168173599
NEWS_AND_MAGAZINES : 2.802893309222423
SOCIAL : 2.667269439421338
TRAVEL_AND_LOCAL : 2.3395117540687163
SHOPPING : 2.2490958408679926
BOOKS_AND_REFERENCE : 2.1360759493670884
DATING : 1.8648282097649187
VIDEO_PLAYERS : 1.7970162748643763
MAPS_AND_NAVIGATION : 1.3901446654611211
FOOD_AND_DRINK : 1.2432188065099457
EDUCATION : 1.164104882459313
ENTERTAINMENT : 0.9606690777576853
LIBRARIES_AND_DEMO : 0.9380650994575045
AUTO_AND_VEHICLES : 0.9267631103074141
HOUSE_AND_HOME : 0.8024412296564195
WEATHER : 0.7911392405063291
EVENTS : 0.7120253164556962
PARENTING : 0.6555153707052441
ART_AND_DESIGN : 0.64

But does a large number of apps make one genre more popular than another? A better indicator of a genre's popularity might be the average number of installs in each genre.

The Android dataset gives us the number of installs for each app; the iOS dataset does not. As a workaround, we'll use the total number of user ratings for each app.

The following code calculates the avarage number of user ratings per app in the iOS dataset.

First, we need to isolate the apps of each genre. We'll start by generating a frequency table for the ```prime_genre``` column of the iOS dataset.

In [28]:
ios_genres = freq_table(ios_free, -5)

Then, we'll loop through the unique genres in the dataset and find the average number of ratings.

In [29]:
for genre in ios_genres:
    total = 0
    len_genre = 0 
    for app in ios_free:
        genre_app = app[-5]
        if genre_app == genre:
            n_ratings = float(app[5])
            total += n_ratings
            len_genre += 1
            
    avg_ratings = total / len_genre
    print(genre, ':', avg_ratings)

Weather : 52279.892857142855
Finance : 32367.02857142857
Travel : 28243.8
Food & Drink : 33333.92307692308
Navigation : 86090.33333333333
Music : 57326.530303030304
Reference : 79350.4705882353
Business : 7491.117647058823
Education : 7003.983050847458
Sports : 23008.898550724636
Medical : 612.0
News : 21248.023255813954
Social Networking : 71548.34905660378
Shopping : 27230.734939759037
Health & Fitness : 23298.015384615384
Lifestyle : 16815.48
Games : 22886.36709539121
Entertainment : 14195.358565737051
Productivity : 21028.410714285714
Catalogs : 4004.0
Photo & Video : 28441.54375
Utilities : 19156.493670886077
Book : 46384.916666666664


Obtaining the average number of installs in the iOS data was a relatively straightforward process. Unfortunately, the Android dataset requires a bit more work. Most install values are open-ended (100+, 1000+, etc.). We don't need perfect precision, so we'll leave the numbers as they are. We just need to remove the plus sign and convert the value to a float value. We can use the ```str.replace()``` method to replace the plus sign and commas with an empty string.

As with the iOS dataset, we'll start by generating a frequency table.

In [32]:
android_categories = freq_table(android_free, 1)

As with the iOS data, we loop through the unique genres, but with the Android data, we can use the number of installs -- after removing the plus sign and commas and converting the value to a float. 

In [37]:
for category in android_categories:
    total = 0
    len_category = 0
    for app in android_free:
        category_app = app[1]
        if category == category_app:
            n_installs = app[5]
            n_installs = n_installs.replace('+', '')
            n_installs = n_installs.replace(',', '')
            n_installs = float(n_installs)
            total += n_installs
            len_category += 1
            
    n_installs = total / len_category
    print(category, ':', n_installs)

ART_AND_DESIGN : 1986335.0877192982
TRAVEL_AND_LOCAL : 13984077.710144928
SPORTS : 3650602.276666667
PHOTOGRAPHY : 17840110.40229885
BUSINESS : 1712290.1474201474
FAMILY : 3695641.8198090694
NEWS_AND_MAGAZINES : 9549178.467741935
DATING : 854028.8303030303
GAME : 15544014.51048951
PERSONALIZATION : 5201482.6122448975
EVENTS : 253542.22222222222
BEAUTY : 513151.88679245283
FOOD_AND_DRINK : 1924897.7363636363
FINANCE : 1387692.475609756
LIFESTYLE : 1446158.2238372094
AUTO_AND_VEHICLES : 647317.8170731707
BOOKS_AND_REFERENCE : 8814199.78835979
HEALTH_AND_FITNESS : 4188821.9853479853
EDUCATION : 1833495.145631068
WEATHER : 5145550.285714285
COMICS : 832613.8888888889
SHOPPING : 7036877.311557789
TOOLS : 10830251.970588235
MAPS_AND_NAVIGATION : 4049274.6341463416
PARENTING : 542603.6206896552
LIBRARIES_AND_DEMO : 638503.734939759
HOUSE_AND_HOME : 1360598.042253521
PRODUCTIVITY : 16787331.344927534
COMMUNICATION : 38590581.08741259
MEDICAL : 120550.61980830671
VIDEO_PLAYERS : 24727872.452830